# Исследование генома

Каждый модуль можно запускать отдельно при наличии нужных файлов

## Подключение нужных ресурсов

In [6]:
# %pip magic
# !pip install hmmer
# /usr/local/bin/python3 -m pip install hmmer

# ls

In [1]:
%%bash
wget http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz
tar xf hmmer-3.3.2.tar.gz
cd hmmer-3.3.2
./configure
make
make install

configure: Configuring HMMER3 for your system.
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking whether to compile using MPI... no
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for gcc option to accept ISO C99... none needed
checking for gcc option to accept ISO Standard C... (cached) none needed
checking how to run the C preprocessor... gcc -E
checking for a BSD-compatible install... /usr/bin/install -c
checking for strip... strip
checking for ranlib... ranlib
checking for ar... /usr/bin/ar
checking whether ln -s works... yes
checking for a sed that does not truncat

--2025-06-16 20:21:32--  http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz
Resolving eddylab.org (eddylab.org)... 96.126.110.11, 2600:3c03::f03c:91ff:fec8:383c
Connecting to eddylab.org (eddylab.org)|96.126.110.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18213049 (17M) [application/x-gzip]
Saving to: ‘hmmer-3.3.2.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  191K 93s
    50K .......... .......... .......... .......... ..........  0%  380K 70s
   100K .......... .......... .......... .......... ..........  0%  139M 46s
   150K .......... .......... .......... .......... ..........  1% 70.7M 35s
   200K .......... .......... .......... .......... ..........  1%  383K 37s
   250K .......... .......... .......... .......... ..........  1% 90.5M 31s
   300K .......... .......... .......... .......... ..........  1% 81.2M 26s
   350K .......... .......... .......... .......... ..........  2% 87.7M 23s
   400K .......... ......

CalledProcessError: Command 'b'wget http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz\ntar xf hmmer-3.3.2.tar.gz\ncd hmmer-3.3.2\n./configure\nmake\nmake install\n'' returned non-zero exit status 2.

In [2]:
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz

--2025-06-16 20:24:10--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293000230 (279M) [application/x-gzip]
Saving to: ‘Pfam-A.hmm.gz’

     0K .......... .......... .......... .......... ..........  0%  470K 10m9s
    50K .......... .......... .......... .......... ..........  0%  929K 7m39s
   100K .......... .......... .......... .......... ..........  0% 86.9M 5m7s
   150K .......... .......... .......... .......... ..........  0%  197M 3m50s
   200K .......... .......... .......... .......... ..........  0%  946K 4m5s
   250K .......... .......... .......... .......... ..........  0%  109M 3m24s
   300K .......... .......... .......... .......... ..........  0% 17.1M 2m57s
   350K .......... .......... .......... .......... ..........  0% 45.1M 2m36s
   400K .........

In [ ]:
!gunzip Pfam-A.hmm.gz
!hmmpress Pfam-A.hmm
!hmmfetch --index Pfam-A.hmm.h3m

In [ ]:
!hmmstat Pfam-A.hmm.h3m | tee pfam.txt

In [ ]:
!hmmstat Pfam-A.hmm.h3m | head -n 20

## Получение списка участков

In [ ]:
accessions = dict()

with open('pfam.txt', 'r') as f:
    for line in f.readlines()[8:]:
        accession = line.split()[2]
        name = accession.split('.')[0]
        if name not in accessions.keys():
            accessions[name] = [accession]
        else:
            accessions[name].append(accession)

In [ ]:
import pandas as pd

pfam_selected = pd.read_csv('pfam-selected.csv')
domains = set()

for info in pfam_selected['Pfam domains']:
    for block in info.split():
        if block.startswith('PF'):
            for name in block.split('PF'):
                if len(name) >= 5:
                    domains.add(f'PF{name[:5]}')

domains = list(domains)
domains[:10]

In [ ]:
selected = list()

for d in domains:
    if d not in accessions.keys():
        continue
    for name in accessions[d]:
        selected.append(name)
selected = list(set(selected))

with open('hmm_selected.txt', 'w') as f:
    for name in selected:
        f.write(f'{name}\n')

len(domains), len(selected)

## Анализ

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/850/265/GCA_902850265.1_PGI_Spodlit_v1/GCA_902850265.1_PGI_Spodlit_v1_protein.faa.gz

In [ ]:
!gunzip /content/GCA_902850265.1_PGI_Spodlit_v1_protein.faa.gz
!mv /content/GCA_902850265.1_PGI_Spodlit_v1_protein.faa protein.faa

In [ ]:
!hmmfetch -f Pfam-A.hmm.h3m hmm_selected.txt > selected.hmm
!hmmsearch selected.hmm protein.faa > result_hmm.txt

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/850/265/GCA_902850265.1_PGI_Spodlit_v1/GCA_902850265.1_PGI_Spodlit_v1_feature_table.txt.gz
!gunzip /content/GCA_902850265.1_PGI_Spodlit_v1_feature_table.txt.gz
!mv /content/GCA_902850265.1_PGI_Spodlit_v1_feature_table.txt feature_table.txt

In [54]:
!sed -i '1d' feature_table.txt
features = pd.read_csv('feature_table.txt', sep='\t', header=None,
                       names=['feature',	'class', 'assembly', 'assembly_unit', 'seq_type',
                              'chromosome', 'genomic_accession', 'start', 'end', 'strand',
                              'product_accession', 'non-redundant_refseq', 'related_accession',
                              'name',	'symbol', 'GeneID', 'locus_tag', 'feature_interval_length',
                              'product_length', 'attributes'])
features

sed: 1: "feature_table.txt": invalid command code f


/var/folders/gn/br3jsvn97px48jrqg8tdrtrm0000gn/T/ipykernel_98648/1668591004.py:2: DtypeWarning: Columns (7,8,11,13,14,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv('feature_table.txt', sep='\t', header=None,


,feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
0,# feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
1,gene,protein_coding,GCA_902850265.1,Primary Assembly,chromosome,1,LR824532.2,2076,6622,-,NaN,NaN,NaN,NaN,NaN,NaN,SPLIT_LOCUS1,4547,NaN,NaN
2,mRNA,NaN,GCA_902850265.1,Primary Assembly,chromosome,1,LR824532.2,2076,6622,-,NaN,NaN,CAD0217583.1,NaN,NaN,NaN,SPLIT_LOCUS1,2328,NaN,NaN
3,CDS,with_protein,GCA_902850265.1,Primary Assembly,chromosome,1,LR824532.2,2190,6406,-,CAD0217583.1,NaN,NaN,NaN,NaN,NaN,SPLIT_LOCUS1,1998,665,NaN
4,gene,protein_coding,GCA_902850265.1,Primary Assembly,chromosome,1,LR824532.2,10058,23036,+,NaN,NaN,NaN,NaN,NaN,NaN,SPLIT_LOCUS2,12979,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39871,mRNA,NaN,GCA_902850265.1,Primary Assembly,chromosome,Z,LR824562.2,21410126,21439209,+,NaN,NaN,CAH1647934.1,NaN,NaN,NaN,SPLIT_LOCUS13279,1634,NaN,NaN
39872,CDS,with_protein,GCA_902850265.1,Primary Assembly,chromosome,Z,LR824562.2,21410267,21439063,+,CAH1647934.1,NaN,NaN,NaN,NaN,NaN,SPLIT_LOCUS13279,1347,448.0,NaN
39873,gene,protein_coding,GCA_902850265.1,Primary Assembly,chromosome,Z,LR824562.2,21449401,21454597,+,NaN,NaN,NaN,NaN,NaN,NaN,SPLIT_LOCUS13280,5197,NaN,NaN
39874,mRNA,NaN,GCA_902850265.1,Primary Assembly,chromosome,Z,LR824562.2,21449401,21454597,+,NaN,NaN,CAH1647935.1,NaN,NaN,NaN,SPLIT_LOCUS13280,1488,NaN,NaN


In [56]:
!head -n 50 result_hmm.txt

# tmpscahf0zt_hmmsearch :: search profile(s) against a sequence database
# HMMER 3.4 (Aug 2023); http://hmmer.org/
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  selected.hmm
# target sequence database:        protein.faa
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       BRCT  [M=79]
Accession:   PF00533.29
Description: BRCA1 C Terminus (BRCT) domain
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence     Description
    ------- ------ -----    ------- ------ -----   ---- --  --------     -----------
      1e-38  131.1   3.8      2e-09   37.2   0.0    7.1  7  CAH1641015.1  unnamed protein product [Spodoptera littoralis]
    1.5e-36  124.2   0.0  

In [58]:
!grep 'Query:' result_hmm.txt | wc -l

     510


In [60]:
results = pd.DataFrame(columns=['checked domain', 'accession', 'domain description',
                                'protein', 'protein description', 'protein coordinates', 'e-value'])

flag = False
ind = 100
with open('result_hmm.txt', 'r') as f:
    for line in f.readlines():
        ind += 1
        if line.startswith('Query:'):
            flag = False
            results.loc[len(results), 'checked domain'] = line.split()[1]
            continue
        if line.startswith('Accession'):
            results.loc[len(results) - 1, 'accession'] = line.split()[1]
            continue
        if line.startswith('Description'):
            results.loc[len(results) - 1, 'domain description'] = line[len('Description:') + 1:-1]
            continue
        if line.startswith('>>') and flag == False:
            protein = line.split()[1]
            description = line[len(f'>> {protein}  '):-1]
            results.loc[len(results) - 1, 'protein'] = protein
            results.loc[len(results) - 1, 'protein description'] = description
            assert((features['product_accession'] == protein).sum() == 1)
            feature = features[features['product_accession'] == protein]
            start = feature.loc[feature.index[0], 'start']
            end = feature.loc[feature.index[0], 'end']
            strand = feature.loc[feature.index[0], 'strand']
            results.loc[len(results) - 1, 'protein coordinates'] = f'{strand}{start}:{end}'
            flag = True
        if line.startswith('    E-value'):
            ind = 0
            continue
        if ind == 2:
            if len(line.split()) < 1:
                continue
            if line.startswith('  ------ inclusion threshold ------'):
                flag = True
                continue
            results.loc[len(results) - 1, 'e-value'] = line.split()[0]

results

,checked domain,accession,domain description,protein,protein description,protein coordinates,e-value
0,BRCT,PF00533.29,BRCA1 C Terminus (BRCT) domain,CAH1641015.1,unnamed protein product [Spodoptera littoralis],+3878060:3896756,1e-38
1,GF_recep_IV,PF14843.9,Growth factor receptor domain IV,CAH1647486.1,unnamed protein product [Spodoptera littoralis],+7033003:7048084,3.5e-40
2,DUF3512,PF12024.11,Domain of unknown function (DUF3512),CAH1639418.1,unnamed protein product [Spodoptera littoralis],+971831:977269,2e-29
3,PRMT5,PF05185.19,PRMT5 arginine-N-methyltransferase,CAH1643567.1,unnamed protein product [Spodoptera littoralis],+10548577:10551757,1.1e-60
4,BPL_C,PF02237.20,Biotin protein ligase C terminal domain,CAH1636078.1,unnamed protein product [Spodoptera littoralis],-6687975:6699096,0.00025
...,...,...,...,...,...,...,...
505,Mad3_BUB1_I,PF08311.15,Mad3/BUB1 homology region 1,CAH1647139.1,unnamed protein product [Spodoptera littoralis],-8680317:8688295,8.3e-40
506,RAG1,PF12940.10,"Recombination-activation protein 1 (RAG1), rec...",NaN,NaN,NaN,NaN
507,DUF908,PF06012.15,Domain of Unknown Function (DUF908),NaN,NaN,NaN,NaN
508,SWIB,PF02201.21,SWIB/MDM2 domain,CAH1641379.1,unnamed protein product [Spodoptera littoralis],-859067:864877,1.8e-24


In [100]:
# results[results.protein.notna()].reset_index()

results[results["checked domain"] == "HLH"]

,checked domain,accession,domain description,protein,protein description,protein coordinates,e-value
447,HLH,PF00010.29,Helix-loop-helix DNA-binding domain,CAH1642715.1,unnamed protein product [Spodoptera littoralis],+3212343:3234844,1.9e-25


In [ ]:
features[features["checked domain"] == "HLH"]

In [64]:
results.to_csv('hmm_results.csv')

## ZDNABERT

In [2]:
# %pip magic
# %pip install --upgrade pip
%pip install --upgrade torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 20.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 16.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 582.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from torch import nn
# import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
# from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

print('final')

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-06-17 12:49:46.455979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

final


In [3]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

In [4]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [5]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'

In [ ]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

Downloading...
From (original): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx
From (redirected): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx&confirm=t&uuid=18258bdd-4100-43ea-8aec-694105106a54
To: /content/pytorch_model.bin
100% 354M/354M [00:05<00:00, 70.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
To: /content/config.json
100% 634/634 [00:00<00:00, 2.15MB/s]
Downloading...
From: https://drive.google.com/uc?id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
To: /content/special_tokens_map.json
100% 112/112 [00:00<00:00, 388kB/s]
Downloading...
From: https://drive.google.com/uc?id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
To: /content/tokenizer_config.json
100% 40.0/40.0 [00:00<00:00, 88.7kB/s]
Downloading...
From: https://drive.google.com/uc?id=1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0
To: /content/vocab.txt
100% 28.7k/28.7k [00:00<00:00, 61.2MB/s]


In [6]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Загружаем данные и предсказываем выводы

In [10]:
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/850/265/GCA_902850265.1_PGI_Spodlit_v1/GCA_902850265.1_PGI_Spodlit_v1_genomic.fna.gz
!gunzip GCA_902850265.1_PGI_Spodlit_v1_genomic.fna.gz
!mv GCA_902850265.1_PGI_Spodlit_v1_genomic.fna genomic.fna

In [7]:
with open('genomic.fna', 'rb') as f:
    uploaded={'genomic': f.read()}

In [ ]:
out = []
for key in uploaded.keys():
    print(key)
    out.append(key)
    result_dict = {}
    for seq_record in SeqIO.parse(StringIO(BytesIO(uploaded[key]).read().decode('UTF-8')), 'fasta'):
        kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
        seq_pieces = split_seq(kmer_seq)
        print(seq_record.name)
        out.append(seq_record.name)
        with torch.no_grad():
            preds = []
            for seq_piece in tqdm(seq_pieces):
                input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
                outputs = torch.softmax(model(input_ids.to(device).unsqueeze(0))[-1],axis = -1)[0,:,1]
                preds.append(outputs.cpu().numpy())
        result_dict[seq_record.name] = stitch_np_seq(preds)



        labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
        print('  start     end')
        out.append('  start     end')
        for label in range(1, max_label+1):
            candidate = np.where(labeled == label)[0]
            candidate_length = candidate.shape[0]
            if candidate_length>minimum_sequence_length:
                # print('{:10}'.format(candidate[0]), '{:10}'.format(candidate[-1]))
                out.append('{:10}'.format(candidate[0]) + '{:10}'.format(candidate[-1]))
        print(seq_record.name, "is done now")

    with open(key + '.preds.pkl',"wb") as fh:
      pickle.dump(result_dict, fh)
    print()

with open('text_predictions_zdnabert.txt',"w") as fh:
    for item in out:
        fh.write("%s\n" % item)

genomic
LR824532.2


100%|██████████| 28341/28341 [12:07<00:00, 38.93it/s]


  start     end
LR824532.2 is done now
LR824533.2


100%|██████████| 34940/34940 [14:55<00:00, 39.00it/s]


  start     end
LR824533.2 is done now
LR824534.2


...

100%|██████████| 30980/30980 [13:10<00:00, 39.17it/s]


  start     end
LR824541.2 is done now
LR824542.2


100%|██████████| 31060/31060 [13:11<00:00, 39.23it/s]


  start     end
LR824542.2 is done now
LR824543.2


100%|██████████| 26383/26383 [11:12<00:00, 39.21it/s]


In [ ]:
files.download('text_predictions_zdnabert.txt')

In [ ]:
import pandas as pd
from tqdm import tqdm

results = pd.DataFrame(columns=['Scaffold', 'Start', 'End'])

with open('text_predictions_zdnabert.txt', 'r') as f:
    scaffold = ''
    lines = f.readlines()
    for line in tqdm(lines):
        if line == 'genomic\n':
            continue
        if line.startswith('NW'):
            scaffold = line[:-1]
            continue
        if line.split()[0] == 'start':
            continue
        if len(line.split()) < 2:
            start = int(line.split()[0][0:8])
            end = int(line.split()[0][8:16])
        else:
            start, end = int(line.split()[0]), int(line.split()[1])
        results.loc[len(results)] = [scaffold, start, end]

results

In [ ]:
results.to_csv('zdnabert_predictions.bed', index=False, header=False, sep='\t')

In [ ]:
files.download('zdnabert_predictions.bed')

## ZHUNT

In [ ]:
%chmod a+x zhunt3-alan.c
%gcc zhunt3-alan.c -lm -o zhunt3

In [ ]:
!./zhunt3 12 8 12 genomic.fna

In [ ]:
!head genomic.fna.Z-SCORE

In [ ]:
seg_dict = {}
sum = 1
input_file = "genomic.fna"
fasta_sequence = SeqIO.parse(input_file,'fasta')

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    with open('lengthes.txt', 'a') as f:
        seg_dict[name] = (sum, sum + len(sequence) - 1)
        sum += len(sequence)
seg_dict

In [ ]:
from tqdm import tqdm

!rm 'results_zhunt.bed'

with open('genomic.fna.Z-SCORE', 'r') as f:
    ind = -1
    while 1:
        ind += 1
        line = f.readline()
        if not line:
            break
        if ind < 1:
            continue
        start = int(line.split()[0])
        end = int(line.split()[1])
        score = float(line.split()[5])
        name_start = ''
        name_end = ''

        if score < 300:
            continue

        for key, (l, r) in seg_dict.items():
            if start >= l and start <= r:
                name_start = key
                start -= l
                if end >= l and end <= r:
                    name_end = key
                    end -= l
                break

        if name_start != name_end:
            continue

        if name_start == '':
            continue

        with open('results_zhunt.bed', 'a') as g:
            g.write(f'{name_start}\t{start}\t{end}\n')

!head 'results_zhunt.bed'

## G-квадруплексы

In [ ]:
import re
from Bio import SeqIO
import numpy as np
import pandas as pd

In [ ]:
input_file = "genomic.fna"
fasta_sequence = SeqIO.parse(input_file,'fasta')

pattern_plus = "((?:G{3,}[ATGC]{1,7}){3,}G{3,})"
pattern_minus = "((?:C{3,}[ATGC]{1,7}){3,}C{3,})"

In [ ]:
import re

results = pd.DataFrame(columns = ['Scaffold', 'Start', 'End', 'Name', 'Score', 'Strand'])

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    PQS_plus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_plus, sequence, re.IGNORECASE)]
    PQS_plus_numpy = np.array(PQS_plus)
    if len(PQS_plus_numpy) == 0:
        continue
    new = pd.DataFrame(data={'Scaffold': [name]*len(PQS_plus_numpy),
                             'Start': PQS_plus_numpy[:,0], 'End': PQS_plus_numpy[:,1],
                             'Name': ['']*len(PQS_plus_numpy), 'Score': ['']*len(PQS_plus_numpy),
                             'Strand': ['+']*len(PQS_plus_numpy)})
    results = pd.concat([results, new], ignore_index=True)

    PQS_minus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_minus, sequence, re.IGNORECASE)]
    PQS_minus_numpy = np.array(PQS_minus)
    if len(PQS_minus_numpy) == 0:
        continue
    new = pd.DataFrame(data={'Scaffold': [name]*len(PQS_minus_numpy),
                             'Start': PQS_minus_numpy[:,0] , 'End': PQS_minus_numpy[:,1],
                             'Name': ['']*len(PQS_minus_numpy), 'Score': ['']*len(PQS_minus_numpy),
                             'Strand': ['-']*len(PQS_minus_numpy)})
    results = pd.concat([results, new], ignore_index=True)
results

In [ ]:
results.to_csv('G-quadruplex.bed', index=False, header=False, sep='\t')

## Анализ

Скачиваем bedtools для jupyter notebook: 

In [16]:
conda install -c bioconda bedtools

Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/htotu/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/htotu/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
Retrieving notices: ...working... done
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::datashader==0.15.2=py311hecd8cb5_0
  - defaults/osx-64::_anaconda_depends==2023.09=py311_openblas_1
  - defaults/osx-64::panel==1.2.3=py311hecd8cb5_0
  - defaults/osx-64::pyviz_comms==2.3.0=py311hecd8cb5_0
  - defaults/osx-64::pyct==0.5.0=py311hecd8cb5_0
  - defaults/osx-64::colorcet==3.0.1=py311hecd8cb5_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 

In [ ]:
# conda update -n base -c defaults conda

In [19]:
import pandas as pd
from Bio import SeqIO

In [35]:
bert_df = pd.read_csv('zdnabert_predictions.bed', header=None, sep='\t',
                   names=['Scaffold', 'Start', 'End'])
bert_df

,Scaffold,Start,End
0,LR824532.2,1069,1080
1,LR824532.2,1143,1160
2,LR824532.2,2257,2269
3,LR824532.2,2366,2380
4,LR824532.2,2399,2411
...,...,...,...
97884,LR824562.2,21487416,21487437
97885,LR824562.2,21488759,21488775
97886,LR824562.2,21489396,21489410
97887,LR824562.2,21490319,21490349


In [37]:
zhunt_df = pd.read_csv('results_zhunt.bed', header=None, sep='\t',
                   names=['Scaffold', 'Start', 'End'])
zhunt_df

,Scaffold,Start,End
0,LR824532.2,1069,1093
1,LR824532.2,1070,1094
2,LR824532.2,1071,1093
3,LR824532.2,1072,1094
4,LR824532.2,1073,1093
...,...,...,...
2117012,LR824562.2,21490735,21490751
2117013,LR824562.2,21490736,21490752
2117014,LR824562.2,21490737,21490753
2117015,LR824562.2,21490738,21490754


In [39]:
gq_df = pd.read_csv('G-quadruplex.bed', header=None, sep='\t',
                 names=['Scaffold', 'Start', 'End', 'Name', 'Score' ,'Strand'])
gq_df

,Scaffold,Start,End,Name,Score,Strand
0,LR824532.2,2283,2348,NaN,NaN,+
1,LR824532.2,20796,20822,NaN,NaN,+
2,LR824532.2,20863,20886,NaN,NaN,+
3,LR824532.2,22972,23014,NaN,NaN,+
4,LR824532.2,48874,48915,NaN,NaN,+
...,...,...,...,...,...,...
23016,LR824562.2,21358037,21358084,NaN,NaN,-
23017,LR824562.2,21395791,21395823,NaN,NaN,-
23018,LR824562.2,21407322,21407366,NaN,NaN,-
23019,LR824562.2,21474736,21474757,NaN,NaN,-


In [9]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/850/265/GCA_902850265.1_PGI_Spodlit_v1/GCA_902850265.1_PGI_Spodlit_v1_genomic.gff.gz
!gunzip GCA_902850265.1_PGI_Spodlit_v1_genomic.gff.gz
!mv GCA_902850265.1_PGI_Spodlit_v1_genomic.gff genomic.gff
!head genomic.gff

--2025-06-18 15:11:55--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/850/265/GCA_902850265.1_PGI_Spodlit_v1/GCA_902850265.1_PGI_Spodlit_v1_genomic.gff.gz
Распознаётся ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Подключение к ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 3689926 (3,5M) [application/x-gzip]
Сохранение в: «GCA_902850265.1_PGI_Spodlit_v1_genomic.gff.gz»

GCA_902850265.1_PGI 100%[===================>]   3,52M   117KB/s    за 17s     

2025-06-18 15:12:13 (214 KB/s) - «GCA_902850265.1_PGI_Spodlit_v1_genomic.gff.gz» сохранён [3689926/3689926]

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build PGI_Spodlit_v1
#!genome-build-accession NCBI_Assembly:GCA_902850265.1
##sequence-region LR824532.2 1 14056742
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=7109
LR824532.2	EMBL	re

In [11]:
with open('genomic.gff', 'r') as f:
    for ind, line in enumerate(f.readlines()):
        if ind < 7:
            continue
        if line.startswith('#'):
            continue
        feature = line.split()[2]
        if not feature in ['mRNA', 'CDS', 'gene', 'exon']:
            continue
        file_name = f'{feature}.bed'
        with open(file_name, 'a') as g:
            add = line.split()[0] + '\t' + line.split()[3] + '\t' + line.split()[4] + '\t' + 'name\tscore\t' + line.split()[6] + '\n'
            g.write(add)

In [13]:
!bedtools subtract -a gene.bed -b exon.bed > intron.bed

In [15]:
!head intron.bed

LR824532.2	2261	2359	name	score	-
LR824532.2	2682	2757	name	score	-
LR824532.2	2957	3122	name	score	-
LR824532.2	4165	4414	name	score	-
LR824532.2	4638	4964	name	score	-
LR824532.2	5001	6313	name	score	-
LR824532.2	10380	19403	name	score	+
LR824532.2	21032	22065	name	score	+
LR824532.2	23857	27280	name	score	-
LR824532.2	27322	42808	name	score	-


In [21]:
input_file = "genomic.fna"

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    with open('lengthes.txt', 'a') as f:
        f.write(f'{name}\t{len(sequence)}\n')

!head lengthes.txt

LR824532.2	14056742
LR824533.2	17330143
LR824534.2	16662782
LR824535.2	16030288
LR824536.2	16049177
LR824537.2	15725378
LR824538.2	15814246
LR824539.2	15990289
LR824540.2	15458409
LR824541.2	15365786


In [23]:
!bedtools flank -l 1000 -r 0 -i gene.bed -g lengthes.txt -s > upstream.bed

In [25]:
!bedtools flank -r 200 -l 0 -i gene.bed -g lengthes.txt -s > downstream.bed

In [27]:
!bedtools complement -i gene.bed -g lengthes.txt > intergenic.bed

In [154]:
results = ['G-quadruplex.bed', 'results_zhunt.bed', 'zdnabert_predictions.bed']

for r in results:
    name = f'new_{r}'
    !cp {r} {name}

In [156]:
places = ['exon', 'intron', 'upstream', 'downstream', 'intergenic']
counts = pd.DataFrame(columns=['G4', 'zhunt', 'zdnabert', 'all'],
                      index = ['exon', 'intron', 'upstream', 'downstream', 'intergenic'])

for p in places:
    name_place = f'{p}.bed'
    for ind, algo in enumerate(['G4', 'zhunt', 'zdnabert']):
        name_res = f'new_{results[ind]}'
        name_out = f'cnt_{p}_{algo}.bed'
        with open(name_res, 'r') as f:
            before = len(f.readlines())
        !bedtools intersect -a {name_res} -b {name_place} -wa > {name_out}
        !bedtools subtract -a {name_res} -b {name_out} -A > tmp.bed
        !rm {name_res}
        !mv tmp.bed {name_res}
        with open(name_res, 'r') as f:
            after = len(f.readlines())
        counts.loc[p, algo] = before - after
    with open(name_place, 'r') as f:
        counts.loc[p, 'all'] = len(f.readlines())
counts.loc['all'] = [len(gq_df), len(zhunt_df), len(bert_df), 'NaN']
counts

,G4,zhunt,zdnabert,all
exon,1254,236986,18562,196115
intron,7039,650192,24592,85644
upstream,415,58473,2668,13277
downstream,114,15300,604,13277
intergenic,14199,1156066,51463,12822
all,23021,2117017,97889,NaN


In [158]:
frac = pd.DataFrame(columns=['G4', 'zhunt', 'zdnabert'],
                    index = [ 'exon', 'intron', 'upstream', 'downstream', 'intergenic'],
                    dtype=float)
for p in places:
    for t in ['G4', 'zhunt', 'zdnabert']:
        frac.loc[p, t] = counts.loc[p, t] / counts.loc['all', t]
frac

,G4,zhunt,zdnabert
exon,0.054472,0.111943,0.189623
intron,0.305764,0.307126,0.251223
upstream,0.018027,0.027620,0.027255
downstream,0.004952,0.007227,0.006170
intergenic,0.616785,0.546083,0.525728


In [47]:
frac.sum()

G4          1.0
zhunt       1.0
zdnabert    1.0
dtype: float64

In [180]:
all_features_1 = counts.copy()

all_features_1["Доля G4"] = 1
all_features_1["Доля zhunt"] = 1
all_features_1["Доля zdnabert"] = 1

for p in places:
    all_features_1.loc[p, "Доля G4"] = all_features_1.loc[p, 'G4'] / all_features_1.loc['all', 'G4']
    all_features_1.loc[p, "Доля zhunt"] = counts.loc[p, 'zhunt'] / all_features_1.loc['all', 'zhunt']
    all_features_1.loc[p, "Доля zdnabert"] = counts.loc[p, 'zdnabert'] / all_features_1.loc['all', 'zdnabert']

all_features_1 = all_features_1.drop(["all"], axis=1)
all_features_1 = all_features_1.rename(columns={"G4": "Количество G4", "zhunt": "Количество zhunt", "zdnabert": "Количество zdnabert"})
all_features_1 = all_features_1[["Количество G4", "Доля G4", "Количество zhunt", "Доля zhunt", "Количество zdnabert", "Доля zdnabert"]]
# all_features_1["G4 %"] = all_features_1["G4"] / all_features_1["all"]

all_features_1

,Количество G4,Доля G4,Количество zhunt,Доля zhunt,Количество zdnabert,Доля zdnabert
exon,1254,0.054472,236986,0.111943,18562,0.189623
intron,7039,0.305764,650192,0.307126,24592,0.251223
upstream,415,0.018027,58473,0.027620,2668,0.027255
downstream,114,0.004952,15300,0.007227,604,0.006170
intergenic,14199,0.616785,1156066,0.546083,51463,0.525728
all,23021,1.000000,2117017,1.000000,97889,1.000000


In [206]:
from tqdm.auto import tqdm

types = ["G4", "zhunt", "zdnabert"]
locs = ["gene", "intron", "upstream", "downstream", "intergenic", "exon"]

all_features_2 = pd.DataFrame(
    columns=list(map(lambda s: "Количество регионов " + s, types)) + \
    list(map(lambda s: "Доля в процентах " + s, types)),
    index=locs)

for loc in tqdm(locs):
    for ty in tqdm(types, leave=False):
        all_count = len(open(loc + ".bed").readlines())
        !bedtools intersect -wa -a {loc + ".bed"} -b {ty + ".bed"} | uniq > tmp
        in_count = len(open("tmp").readlines())
        all_features_2.loc[loc, "Количество регионов " + ty] = in_count
        all_features_2.loc[loc, "Доля в процентах " + ty] = in_count / all_count

all_features_2

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Количество регионов G4,Количество регионов zhunt,Количество регионов zdnabert,Доля в процентах G4,Доля в процентах zhunt,Доля в процентах zdnabert
gene,3947,9478,7347,0.297281,0.713866,0.553363
intron,5567,25631,10881,0.065002,0.299274,0.127049
upstream,456,4381,2055,0.034345,0.329969,0.154779
downstream,125,1322,557,0.009415,0.099571,0.041952
intergenic,3869,8776,6495,0.301747,0.684449,0.506551
exon,1505,19735,19368,0.007674,0.10063,0.098758


In [208]:
all_features_2 = all_features_2[["Количество регионов G4", "Доля в процентах G4", "Количество регионов zhunt", "Доля в процентах zhunt", "Количество регионов zdnabert", "Доля в процентах zdnabert"]]

all_features_2

,Количество регионов G4,Доля в процентах G4,Количество регионов zhunt,Доля в процентах zhunt,Количество регионов zdnabert,Доля в процентах zdnabert
gene,3947,0.297281,9478,0.713866,7347,0.553363
intron,5567,0.065002,25631,0.299274,10881,0.127049
upstream,456,0.034345,4381,0.329969,2055,0.154779
downstream,125,0.009415,1322,0.099571,557,0.041952
intergenic,3869,0.301747,8776,0.684449,6495,0.506551
exon,1505,0.007674,19735,0.10063,19368,0.098758
